In [1]:
from pathlib import Path

data_dir = Path.cwd() / "data" / "uncompressed"

mimiic_dir = data_dir / "mimiciv" / "3.1"
eicu_dir = data_dir / "eicu-crd" / "2.0"

In [2]:
import dask.dataframe as dd

from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 47.05 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45269,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:33999,Total threads: 4
Dashboard: http://127.0.0.1:45725/status,Memory: 11.76 GiB
Nanny: tcp://127.0.0.1:43061,


2025-08-31 16:26:16,974 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle a64ac011887f7322620dc155e8aa4126 initialized by task ('shuffle-transfer-a64ac011887f7322620dc155e8aa4126', 0) executed on worker tcp://127.0.0.1:40799
2025-08-31 16:26:17,634 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle a64ac011887f7322620dc155e8aa4126 deactivated due to stimulus 'task-finished-1756657577.6331983'
2025-08-31 16:26:18,789 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 8d2dd96c9f06cf575233b71876dd5cdd initialized by task ('shuffle-transfer-8d2dd96c9f06cf575233b71876dd5cdd', 0) executed on worker tcp://127.0.0.1:33999
2025-08-31 16:26:18,824 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 8d2dd96c9f06cf575233b71876dd5cdd deactivated due to stimulus 'task-finished-1756657578.8228052'


In [3]:
from open_icu.config.utils import load_yaml_configs
from open_icu.config.source import SourceConfig

configs = load_yaml_configs(Path.cwd().parent / "configs" / "source", SourceConfig)
configs

[SourceConfig(name='mimic', version='3.1', tables=[TableConfig(name='medications', path='icu/inputevents_small.csv', fields=[FieldConfig(field='subject_id', type='int64'), FieldConfig(field='endtime', type='datetime'), FieldConfig(field='amount', type='float32'), FieldConfig(field='amountuom', type='string'), FieldConfig(field='starttime', type='datetime'), FieldConfig(field='rateuom', type='string'), FieldConfig(field='rate', type='float32'), FieldConfig(field='itemid', type='int64')], join=[JoinConfig(name='d_items', path='icu/d_items.csv', fields=[FieldConfig(field='itemid', type='int64'), FieldConfig(field='label', type='string')], both_on=['itemid'], left_on=[], right_on=[], how='left')], events=[EventConfig(name='dosage', fields=MEDSFieldsConfig(subject_id='subject_id', time='endtime', code=['label', 'amountuom'], numeric_value='amount', text_value=None)), EventConfig(name='rate', fields=MEDSFieldsConfig(subject_id='subject_id', time='starttime', code=['label', 'rateuom'], numeri

In [4]:
from open_icu.meds.processor import process_table

process_table(configs[0].tables[0], mimiic_dir, Path.cwd() / "output")

/workspaces/.venv/lib/python3.13/site-packages/dask/dataframe/dask_expr/_collection.py:3250: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))
/workspaces/.venv/lib/python3.13/site-packages/dask/dataframe/dask_expr/_collection.py:3250: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'obj

In [6]:
dd.read_parquet(Path.cwd() / "output" / "data" / "medications_dosage_0.parquet").head()

,subject_id,time,numeric_value,text_value,code
0,10000032,2180-07-23 17:01:00,200.0,,PO Intake//mL
1,10000032,2180-07-23 17:30:00,50.0,,Albumin 25%//mL
2,10000032,2180-07-23 18:03:00,50.0,,Albumin 25%//mL
3,10000032,2180-07-23 18:57:00,100.0,,PO Intake//mL
4,10000032,2180-07-23 21:11:00,100.0,,PO Intake//mL
